In [2]:
import pandas as pd
import os

In [3]:
# Make main and second transcript id input for classify isoform differences script
df = pd.read_table('../data/classify_isoform_differences/LCL_espresso_pairwise_transcript_comparison_2024-08-27.tsv')
df = df[df['FDR'].lt(0.05)]

with open('../data/classify_isoform_differences/main_transcript_id.txt', 'w') as file:
    for id in df['transcript1']:
        file.write(f'  - {id}\n')

with open('../data/classify_isoform_differences/second_transcript_id.txt', 'w') as file:
    for id in df['transcript2']:
        file.write(f'  - {id}\n')

In [ ]:
# After running classify_isoform_differences.py
# Merge isoform difference results into one file
def concatenate_files(files, output_file):
    """Concatenate a list of files but only write the header once."""
    with open(output_file, 'w') as outfile:
        header_written = False
        
        for file in files:
            with open(file, 'r') as infile:
                lines = infile.readlines()
                
                if not header_written:
                    outfile.write(lines[0])
                    header_written = True
                
                for line in lines[1:]:
                    outfile.write(line)

indir = '../results/classify_isoform_differences'
files = os.listdir(indir)
files = [os.path.join(indir, file) for file in files]
outfname = '../results/LCL_isoform_differences.tsv'

concatenate_files(files, outfname)

# Split coordinates column
df = pd.read_table(outfname)
df['coordinates'] = df['coordinates'].str.split(';')
df = df.explode('coordinates')
df['chr'] = df['coordinates'].str.split(':').str[0]
df['start'] = df['coordinates'].str.split(':').str[1]
df['end'] = df['coordinates'].str.split(':').str[2]
df['strand'] = df['coordinates'].str.split(':').str[3]
df.to_csv('../results/LCL_isoform_differences_exploded_coordinates.tsv',
          index=False, sep='\t')